In [1]:
!nvidia-smi

Tue Feb 22 08:41:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 470.63.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:08:00.0 Off |                    0 |
| N/A   25C    P0    32W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:0B:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import os
import csv
import numpy as np
from PIL import Image
import batch_preparation
import keras 

In [4]:
model_folder = 'models_220221'

In [5]:
data_folder = '/media/QNAP/People/BartLiefers/ROP/220118/ROP/Images/Media'

with open('../src/data/new_data_split.csv', 'r') as f:
    split = list(csv.reader(f))
    header = split[0]
    data = split[1:]

In [6]:
header

['file', 'subset', 'CH', 'JT', 'CK', 'SB', 'GA', 'group']

In [7]:
grading = {row[0]: row[-1] for row in data}

In [8]:
all_files = [row[0] for row in data]

In [12]:
#test_files = [row[0] for row in data if row[1] == 'test']

all_data_preprocessed = [(grading[path], batch_preparation.sample_patch(all_data[path], augment=False))
 for path in all_files]

In [9]:
best_models = {fold: max((m.split('_')[-1]) for m in os.listdir(model_folder) if m.split('_')[3] == str(fold))
               for fold in range(5)}
best_models

{0: '77', 1: '71', 2: '90', 3: '95', 4: '95'}

In [10]:
models = {
    fold: keras.models.load_model('{}/val_fold_fold_{}_epoch_{}'.format(model_folder, fold, best_models[fold]))
    for fold in range(5)
}

In [13]:
results = []

for file in all_files:
    path = os.path.join(data_folder, file)
    img = np.array(Image.open(os.path.join(data_folder, path)))[:,:,:3]
    x = batch_preparation.sample_patch(img, augment=False)
    
    for fold in range(5):
        p = models[fold].predict_on_batch(x[np.newaxis])[0]
        results.append((file, fold, *p))

In [14]:
header_out = 'file', 'model', 'normal', 'pre-plus', 'plus'

In [15]:
with open('model220221_classification_all_files.csv', 'w') as f:
    csv.writer(f).writerows([header_out] + results)